In [ ]:
import uuid

folderName = uuid.uuid4()

In [11]:
%%pyspark
# data lake path for container
adls_path = 'abfss://tokyo-olympic-data@tokyoolympicportfolio.dfs.core.windows.net/'

# Defines a list of file names to be read from the ADLS
fileName = ['Athletes','Coaches','EntriesGender', 'Medals', 'Teams']

# Iterates through each file name in the list &
# Reads the CSV file into a Spark DataFrame and assigns it to a global variable named after the file
for name in fileName:
    globals()[name] = spark.read.csv(adls_path + 'raw_data/' + name + '.csv', header=True, inferSchema=True)

StatementMeta(tokyopool, 10, 12, Finished, Available, Finished)

In [12]:
# to preview some files and individual schema
Athletes.show(5)

StatementMeta(tokyopool, 10, 13, Finished, Available, Finished)

+-----------------+------+-------------------+
|             Name|   NOC|         Discipline|
+-----------------+------+-------------------+
|  AALERUD Katrine|Norway|       Cycling Road|
|      ABAD Nestor| Spain|Artistic Gymnastics|
|ABAGNALE Giovanni| Italy|             Rowing|
|   ABALDE Alberto| Spain|         Basketball|
|    ABALDE Tamara| Spain|         Basketball|
+-----------------+------+-------------------+
only showing top 5 rows



In [13]:
Athletes.printSchema()

StatementMeta(tokyopool, 10, 14, Finished, Available, Finished)

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [14]:
Medals.show(5)

StatementMeta(tokyopool, 10, 15, Finished, Available, Finished)

+----+--------------------+----+------+------+-----+-------------+
|Rank|            Team/NOC|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
+----+--------------------+----+------+------+-----+-------------+
only showing top 5 rows



In [15]:
Medals.printSchema()

StatementMeta(tokyopool, 10, 16, Finished, Available, Finished)

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [16]:
# to rename column
Medals = Medals.withColumnRenamed('Team/Noc', 'Team')
Medals.show(2)

StatementMeta(tokyopool, 10, 17, Finished, Available, Finished)

+----+--------------------+----+------+------+-----+-------------+
|Rank|                Team|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
+----+--------------------+----+------+------+-----+-------------+
only showing top 2 rows



In [ ]:
# I created another loop to save the transformed files and also to rename the column 'NOC' to 'Country'
# Loops are fun when I have to iterate a list. I guess I am lazy
for tables in fileName:
    df = globals()[tables]
    globals()[tables] = df.withColumnRenamed("NOC", "Country")
    transformedTable = globals()[tables]
    transformedTable.repartition(1).write.mode('overwrite').option('header', 'true').csv(adls_path + 'transformed_data/' + tables)

In [19]:
# to be sure the code is working
Teams.show(5)

StatementMeta(tokyopool, 10, 20, Finished, Available, Finished)

+-------+--------------+--------------------+-----+
|   Name|    Discipline|             Country|Event|
+-------+--------------+--------------------+-----+
|Belgium|3x3 Basketball|             Belgium|  Men|
|  China|3x3 Basketball|People's Republic...|  Men|
|  China|3x3 Basketball|People's Republic...|Women|
| France|3x3 Basketball|              France|Women|
|  Italy|3x3 Basketball|               Italy|Women|
+-------+--------------+--------------------+-----+
only showing top 5 rows

